In [ ]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TextVectorization


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Import data as pandas datafames
train_data = pd.read_csv(
    train_file_path,
    sep='\t',
    header=None,
    names=['classification', 'text_message']
    )

test_data = pd.read_csv(
    test_file_path,
    sep='\t',
    header=None,
    names=['classification', 'text_message']
    )

print(train_data.head())


In [ ]:
# Make new variables to keep original data intact and assign 0 and 1 to ham and spam.
train_processed = train_data.copy()
train_processed['classification'] = train_processed['classification'].map({'ham': 0, 'spam': 1})

test_processed = test_data.copy()
test_processed['classification'] = test_processed['classification'].map({'ham': 0, 'spam': 1})




In [ ]:
# Make dataframes into tensor datasets
BUFFER_SIZE = 10000
BATCH_SIZE = 32

train_ds = tf.data.Dataset.from_tensor_slices((train_processed['text_message'].values, train_processed['classification'].values))
test_ds = tf.data.Dataset.from_tensor_slices((test_processed['text_message'].values, test_processed['classification'].values))


# Batch datasets and shuffle the training set

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Make a vectorization layer that'll turn the text into numpy arrays for the model
VOCAB_SIZE=100000
vectorize_layer = TextVectorization(
    max_tokens=VOCAB_SIZE)


In [ ]:
# Adapt the vectorize layer to our training data
vectorize_layer.adapt(train_ds.map(lambda text, label: text))

# Print the vocabulary of our vectorization layer
print(vectorize_layer.get_vocabulary())

In [ ]:
# Create model with the vectorization layer, an embedding layer,
# an LSTM layer and two dense layers, the last one being a binary one squished with a sigmoid function
model = tf.keras.Sequential([
    vectorize_layer,
    layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    layers.Bidirectional(tf.keras.layers.LSTM(64)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# Train the model with our training data

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=test_ds,
    validation_steps=30
    )

In [ ]:
# predict something to see if it works
prediction = model.predict(np.array(['how are you doing today?']))
print(prediction)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  # predict the text that was input after converting it to a numpy array
  result = model.predict(np.array([pred_text]))
  print(result[0][0])

  # Classify as ham or spam depending on output
  if result[0][0] >= 0.5:
    classification = 'spam'
  else:
    classification = 'ham'

  # Put output and the text inside an array for returning
  prediction = [result[0][0], classification]


  # Return the prediction
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
